In [2]:
%load_ext autoreload
%autoreload 2

# Feature Pipeline: Computing features

In [ ]:
import os
import sys

repo_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(repo_path)

import torch
import warnings
import polars as pl

from pprint import pprint
from loguru import logger
from sentence_transformers import SentenceTransformer

# Config
from recsys.config import settings

# Feature Store
# from recsys import gcp_integrations
from recsys.gcp_integrations import feature_store, bq_utils

# Features
from recsys.features.articles import (
    compute_features_articles,
    generate_embeddings_for_dataframe,
)
from recsys.features.customers import DatasetSampler, compute_features_customers
from recsys.features.interaction import generate_interaction_data
from recsys.features.transactions import compute_features_transactions

# Raw Data
from recsys.raw_data_sources import h_and_m_raw_data

warnings.filterwarnings("ignore")

# 🗄️ Articles data

The **article_id** and **product_code** serve different purposes in the context of H&M's product database:

- **Article ID**: This is a unique identifier assigned to each individual article within the database. It is typically used for internal tracking and management purposes. Each distinct item or variant of a product (e.g., different sizes or colors) would have its own unique article_id.

- **Product Code**: This is also a unique identifier, but it is associated with a specific product or style rather than individual articles. It represents a broader category or type of product within H&M's inventory. Multiple articles may share the same product code if they belong to the same product line or style.

While both are unique identifiers, the article_id is specific to individual items, whereas the product_code represents a broader category or style of product.

Here is an example:

**Product: Basic T-Shirt**

- **Product Code:** TS001

- **Article IDs:**
    - Article ID: 1001 (Size: Small, Color: White)
    - Article ID: 1002 (Size: Medium, Color: White)
    - Article ID: 1003 (Size: Large, Color: White)
    - Article ID: 1004 (Size: Small, Color: Black)
    - Article ID: 1005 (Size: Medium, Color: Black)

In this example, "TS001" is the product code for the basic t-shirt style. Each variant of this t-shirt (e.g., different sizes and colors) has its own unique article_id.



In [4]:
articles_df = h_and_m_raw_data.extract_articles_df()
articles_df.shape

(105542, 25)

In [5]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,detail_desc
i64,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,str
108775015,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775044,108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"
108775051,108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""","""Jersey top with narrow shoulde…"


## Articles feature engineering

In [6]:
articles_df = compute_features_articles(articles_df)
articles_df.shape

(105542, 27)

In [7]:
articles_df.head(3)

article_id,product_code,prod_name,product_type_no,product_type_name,product_group_name,graphical_appearance_no,graphical_appearance_name,colour_group_code,colour_group_name,perceived_colour_value_id,perceived_colour_value_name,perceived_colour_master_id,perceived_colour_master_name,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name,prod_name_length,article_description,image_url
str,i64,str,i64,str,str,i64,str,i64,str,i64,str,i64,str,i64,str,str,str,i64,str,i64,str,i64,str,u32,str,str
"""108775015""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",9,"""Black""",4,"""Dark""",5,"""Black""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""gs://gonzo-recsys-data/h-and-m…"
"""108775044""",108775,"""Strap top""",253,"""Vest top""","""Garment Upper body""",1010016,"""Solid""",10,"""White""",3,"""Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",9,"""Strap top - Vest top in Garmen…","""gs://gonzo-recsys-data/h-and-m…"
"""108775051""",108775,"""Strap top (1)""",253,"""Vest top""","""Garment Upper body""",1010017,"""Stripe""",11,"""Off White""",1,"""Dusty Light""",9,"""White""",1676,"""Jersey Basic""","""A""","""Ladieswear""",1,"""Ladieswear""",16,"""Womens Everyday Basics""",1002,"""Jersey Basic""",13,"""Strap top (1) - Vest top in Ga…","""gs://gonzo-recsys-data/h-and-m…"


## Create embeddings from the articles description

In [8]:
for i, desc in enumerate(articles_df["article_description"].head(n=3)):
    logger.info(f"Item {i + 1}:\n{desc}")

2025-02-06 10:33:25.018 | INFO     | __main__:<module>:2 - Item 1:
Strap top - Vest top in Garment Upper body
 Apperance: Solid
 Color: Dark Black 9
 Category: Ladieswear Womens Everyday Basics Jersey Basic
 Details: Jersey top with narrow shoulder straps.
2025-02-06 10:33:25.018 | INFO     | __main__:<module>:2 - Item 2:
Strap top - Vest top in Garment Upper body
 Apperance: Solid
 Color: Light White 10
 Category: Ladieswear Womens Everyday Basics Jersey Basic
 Details: Jersey top with narrow shoulder straps.
2025-02-06 10:33:25.019 | INFO     | __main__:<module>:2 - Item 3:
Strap top (1) - Vest top in Garment Upper body
 Apperance: Stripe
 Color: Dusty Light White 11
 Category: Ladieswear Womens Everyday Basics Jersey Basic
 Details: Jersey top with narrow shoulder straps.


In [9]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

logger.info(
    f"Loading ${settings.FEATURES_EMBEDDING_MODEL_ID} embedding model to {device=}"
)

# Load embedding model from SentenceTransformers model registry
model = SentenceTransformer(settings.FEATURES_EMBEDDING_MODEL_ID, device=device)

2025-02-06 10:33:25.053 | INFO     | __main__:<module>:9 - Loading $all-MiniLM-L6-v2 embedding model to device='mps'


In [10]:
articles_df = generate_embeddings_for_dataframe(
    articles_df, "article_description", model, batch_size=128
)

Generating embeddings...: 100%|██████████| 105542/105542 [05:46<00:00, 305.03it/s]


For each article description, we have a numerical vector which we can feed to a model, opposite to a string containing the description of an object.

In [11]:
articles_df[["article_description", "embeddings"]].head(3)

article_description,embeddings
str,list[f64]
"""Strap top - Vest top in Garmen…","[-0.023064, 0.069894, … 0.012397]"
"""Strap top - Vest top in Garmen…","[-0.011304, 0.075073, … 0.016485]"
"""Strap top (1) - Vest top in Ga…","[-0.033244, 0.081816, … 0.020111]"


In [12]:
articles_df["image_url"][0]

'gs://gonzo-recsys-data/h-and-m/images/010/0108775015.jpg'

In [13]:
from IPython.display import HTML, display

image_urls = articles_df["image_url"].tail(12).to_list()
grid_html = '<div style="display: grid; grid-template-columns: repeat(6, 1fr); gap: 10pxl max-width: 900px;">'

for url in image_urls:
    grid_html += f'<img src="{url}" style="width: 100%; height: auto;">'

grid_html += "</div>"

display(HTML(grid_html))

# Customers Data

In [14]:
customers_df = h_and_m_raw_data.extract_customers_df()
customers_df.shape

(1371980, 7)

In [15]:
customers_df.head(3)

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"


In [16]:
customers_df.null_count()

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
u32,u32,u32,u32,u32,u32,u32
0,895050,907576,6062,16009,15861,0


## Customers feature engineering

In [17]:
customers_df.head(3)

customer_id,FN,Active,club_member_status,fashion_news_frequency,age,postal_code
str,f64,f64,str,str,i64,str
"""00000dbacae5abe5e23885899a1fa4…",null,null,"""ACTIVE""","""NONE""",49,"""52043ee2162cf5aa7ee79974281641…"
"""0000423b00ade91418cceaf3b26c6a…",null,null,"""ACTIVE""","""NONE""",25,"""2973abc54daa8a5f8ccfe9362140c6…"
"""000058a12d5b43e67d225668fa1f8d…",null,null,"""ACTIVE""","""NONE""",24,"""64f17e6a330a85798e4998f62d0930…"


In [18]:
customers_df = compute_features_customers(customers_df, drop_null_age=True)
customers_df.shape

(1356119, 5)

In [19]:
customers_df.head(3)

customer_id,club_member_status,age,postal_code,age_group
str,str,f64,str,str
"""00000dbacae5abe5e23885899a1fa4…","""ACTIVE""",49.0,"""52043ee2162cf5aa7ee79974281641…","""46-55"""
"""0000423b00ade91418cceaf3b26c6a…","""ACTIVE""",25.0,"""2973abc54daa8a5f8ccfe9362140c6…","""19-25"""
"""000058a12d5b43e67d225668fa1f8d…","""ACTIVE""",24.0,"""64f17e6a330a85798e4998f62d0930…","""19-25"""


# Transactions Data

In [20]:
transactions_df = h_and_m_raw_data.extract_transactions_df()
transactions_df.shape

(31788324, 5)

In [21]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id
date,str,i64,f64,i64
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",663713001,0.050831,2
2018-09-20,"""000058a12d5b43e67d225668fa1f8d…",541518023,0.030492,2
2018-09-20,"""00007d2de826758b65a93dd24ce629…",505221004,0.015237,2


## Transactions feature engineering

In [22]:
transactions_df = compute_features_transactions(transactions_df)
transactions_df.shape

(31788324, 9)

The time of the year a purchase was made should be a strong predictor, as seasonality plays a big factor in fashion purchases. Here, you will use the month of the purchase as a feature. Since this is a cyclical feature (January is as close to December as it is to February), you'll map each month to the unit circle using sine and cosine.

Thus, the features of the transactions DataFrame look as follows:

In [23]:
transactions_df.head(3)

t_dat,customer_id,article_id,price,sales_channel_id,year,month,day,day_of_week
i64,str,str,f64,i64,i32,i8,i8,i8
0,"""000058a12d5b43e67d225668fa1f8d…","""663713001""",0.050831,2,2018,9,20,4
0,"""000058a12d5b43e67d225668fa1f8d…","""541518023""",0.030492,2,2018,9,20,4
0,"""00007d2de826758b65a93dd24ce629…","""505221004""",0.015237,2,2018,9,20,4


We don't want to work with ~30 million transactions in these series, as everything will take too much time to run. Thus, we create a subset of the original dataset by randomly sampling from the customers' datasets and taking only their transactions.

In [24]:
sampler = DatasetSampler(size=settings.CUSTOMER_DATA_SIZE)
dataset_subset = sampler.sample(
    customers_df=customers_df, transactions_df=transactions_df
)
customers_df = dataset_subset["customers"]
transactions_df = dataset_subset["transactions"]

2025-02-06 10:47:13.548 | INFO     | recsys.features.customers:sample:29 - Sampling 1000 customers.
2025-02-06 10:47:13.586 | INFO     | recsys.features.customers:sample:32 - Number of transactions for all the customers: 31788324
2025-02-06 10:47:14.166 | INFO     | recsys.features.customers:sample:38 - Number of transactions for the 1000 sampled customers: 23799


In [25]:
transactions_df.shape

(23799, 9)

Some of the remaining customers:

In [26]:
for customer_id in transactions_df["customer_id"].unique().head(10):
    logger.info(f"Logging customer ID: {customer_id}")

2025-02-06 10:47:14.275 | INFO     | __main__:<module>:2 - Logging customer ID: 042012178b0189c9f2269bba579273aaded65c23b7aa4e42a3a77b536d3c0403
2025-02-06 10:47:14.275 | INFO     | __main__:<module>:2 - Logging customer ID: c9d80530b826edfbe2b38e97eb47bdee5c24af5a6133727df348f49bd37f9a70
2025-02-06 10:47:14.275 | INFO     | __main__:<module>:2 - Logging customer ID: b34f33be48d6aeab8367b832418f26a83c2ac0dc44e0cb64b75cdceb6f492a6b
2025-02-06 10:47:14.275 | INFO     | __main__:<module>:2 - Logging customer ID: 876feebe6fef4c8ea7493b54b3fa47cf87a132f8b572fa8ca7738210933357f4
2025-02-06 10:47:14.275 | INFO     | __main__:<module>:2 - Logging customer ID: 81d78af25900c010ad78ffe55c5164a7a687b67afc6342bd8dfaffdf9ba06efd
2025-02-06 10:47:14.276 | INFO     | __main__:<module>:2 - Logging customer ID: 409a318a5cc9e51041625286b128443146ed4984b91dee93968d26202bb0e3a8
2025-02-06 10:47:14.276 | INFO     | __main__:<module>:2 - Logging customer ID: dfda540760f5363017a6eb0ae16d983bd5a5374dcb5031a1a1

# 🤳🏻 Interaction data

To train our models, we need more than just the transactions DataFrame. We need positive samples that signal whether a customer clicked or bought an item, but we also need negative samples that signal no interactions between a customer and an item.

In [27]:
interaction_df = generate_interaction_data(transactions_df)
interaction_df.shape

Processing customer chunks:   0%|          | 0/1 [00:03<?, ?it/s]


(135581, 5)

In [28]:
interaction_df.head()

t_dat,customer_id,article_id,interaction_score,prev_article_id
i64,str,str,i64,str
-343800000,"""00b203a32faa3d007dba198ef27c15…","""567418029""",0,"""START"""
-335160000,"""00b203a32faa3d007dba198ef27c15…","""858350001""",0,"""567418029"""
-330480000,"""00b203a32faa3d007dba198ef27c15…","""766439001""",0,"""858350001"""
-329760000,"""00b203a32faa3d007dba198ef27c15…","""766439001""",0,"""766439001"""
-329400000,"""00b203a32faa3d007dba198ef27c15…","""706443001""",0,"""766439001"""


Score distributions:

Here is what each score means:
- `0` : No interaction between a customer and an item
- `1` : A customer clicked an item
- `2` : A customer bought an item

In [29]:
interaction_df.group_by("interaction_score").agg(
    pl.count("interaction_score").alias("total_interactions")
)

interaction_score,total_interactions
i64,u32
0,73459
1,38323
2,23799


# Upload feature group

### Customers

In [60]:
customers_df.head()

customer_id,club_member_status,age,postal_code,age_group
str,str,f64,str,str
"""6f173c0d871007b835f77d16232bae…","""ACTIVE""",61.0,"""aacae4bc0b6cf9c6708b05f234512a…","""56-65"""
"""1ae5ddce4cdf0f21ae41379a6edad9…","""ACTIVE""",23.0,"""2c29ae653a9282cce4151bd87643c9…","""19-25"""
"""b903a2e7b1fba1924a2332c1b69fc4…","""ACTIVE""",25.0,"""612ae24e8d0ea791da6b5b4de91714…","""19-25"""
"""f17d07ee3b52dc06ba23e5dbd0621a…","""ACTIVE""",33.0,"""caaf654c6a82da724a42162a80f037…","""26-35"""
"""27e13eb33e457d98f0f7fd086231a4…","""ACTIVE""",35.0,"""f919da058a4a967dd830c3cbb3c657…","""26-35"""


In [ ]:
logger.info("Uploading 'customers' Feature to BigQuery.")
feature_store.load_features_to_bigquery(df=customers_df)
logger.info("✅ Uploaded 'customers' Feature to BigQuery!")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-02-06 12:46:56.957 | INFO     | recsys.gcp_integrations.feature_store:upload_dataframe_to_bigquery:96 - Uploading DataFrame to BigQuery table: recsys-dev-gonzo.recsys_dataset.recsys_customers
2025-02-06 12:46:56.958 | INFO     | recsys.gcp_integrations.feature_store:upload_dataframe_to_bigquery:100 - Converting Polars DataFrame to Pandas
2025-02-06 12:46:57.001 | INFO     | recsys.gcp_integrations.feature_store:upload_dataframe_to_bigquery:105 - DataFrame shape: (1000, 5)
2025-02-06 12:47:04.367 | INFO     | recsys.gcp_integrations.feature_store:upload_dataframe_to_bigquery:124 - Loaded 1000 rows and 5 columns to recsys-dev-gonzo.recsys_dataset.recsys_customers


### Articles

In [74]:
logger.info("Uploading 'articles' Feature to BigQuery.")
feature_store.load_features_to_bigquery(articles_df=articles_df)
logger.info("✅ Uploaded 'articles' Feature to BigQuery!")

2025-02-06 13:16:12.364 | INFO     | __main__:<module>:1 - Uploading 'articles' Feature to BigQuery.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2025-02-06 13:16:12.686 | INFO     | recsys.gcp_integrations.feature_store:upload_dataframe_to_bigquery:138 - Uploading DataFrame to recsys-dev-gonzo.recsys_dataset.recsys_articles
2025-02-06 13:16:12.686 | INFO     | recsys.gcp_integrations.feature_store:upload_dataframe_to_bigquery:139 - Primary keys: ['article_id']
2025-02-06 13:16:12.795 | ERROR    | recsys.gcp_integrations.feature_store:get_table_schema:102 - Error loading schema for recsys_articles: SchemaField.__init__() got an unexpected keyword argument 'type'
2025-02-06 13:16:12.795 | ERROR    | recsys.gcp_in

TypeError: SchemaField.__init__() got an unexpected keyword argument 'type'

### Transactions

### Interactions

# Compute Ranking Dataset